In [4]:
import tensorflow as tf
from keras import layers

In [5]:
from veggideas.transfer import load_non_trainable_model
from keras import regularizers
from keras import models
from keras import optimizers
from veggideas.load_data import load_train_data, load_val_data

Found 15000 files belonging to 15 classes.
Loading training data
Training data successfully loaded ✅
Found 3000 files belonging to 15 classes.
Loading validation data
Validation data successfully loaded ✅
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                      

KeyboardInterrupt: 

In [ ]:
train_data = load_train_data()
val_data = load_val_data()
model = load_non_trainable_model()

In [ ]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    resize_and_rescale = tf.keras.Sequential([
        layers.Rescaling(1./255)])

    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        layers.RandomBrightness(factor=0.2),
        layers.RandomZoom(.5, .2),
        layers.RandomContrast(factor=0.5),
        ])

    base_model = model
    flattening_layer = layers.Flatten()
    dense_layer = layers.Dense(128, activation='relu')
    dense_layer_2 = layers.Dense(64, activation='relu')
    reg_l2 = regularizers.L2(0.01)
    dense_layer_reg = layers.Dense(128, activation='relu', bias_regularizer=reg_l2)
    maxpool_layer = layers.MaxPool2D(pool_size=(2,2))
    prediction_layer = layers.Dense(15, activation='softmax')

    model = models.Sequential([
        resize_and_rescale,
        data_augmentation,
        base_model,
        flattening_layer,
        dense_layer,
        dense_layer_reg,
        dense_layer_2,
        prediction_layer
        ])


    opt = optimizers.Adam(learning_rate=1e-3)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model


model_transfer = add_last_layers(model)


model_transfer.fit(train_data, batch_size=32, epochs=1, validation_data=val_data)

In [ ]:
def preprocessing_image():
    resize_and_rescale = tf.keras.Sequential([
        layers.Rescaling(1./255)
    ])
    
    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2),
        ])
    return resize_and_rescale, data_augmentation